In [8]:
## This was an attempt to get a unique id for both the authorization and red team table. 
#This did not make it into my final sample because when I ran the new table in databricks 
#for a sanity check I found that when I grouped the data the ID was not unique to both sets. 
#I found that there was not enough data on the red team to make it unique, and if I just added
#a numeric ID to the AUthorization table, I would not be able to join the data that I wanted. 

#import dependencies

from pyspark.sql import functions as sf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,BooleanType

spark = SparkSession.builder.appName("sample authorization table").getOrCreate()

In [ ]:
#Since there isn't a schema for the data we need to manually create it from the information we got from LANL
#Got this from https://www.youtube.com/watch?v=K14plpZgy_c approx 19:45


# Read in data and store in dataframe using authSchema
df = spark.read.csv("sample3.txt",inferSchema=True).toDF("Time","SourceUserDomain","DestinationUserDomain",
                                                        "SourceComputer","DestinationComputer","AuthenticationType",
                                                        "LogonType","AuthenticationOrientation","SuccessFailure")

In [10]:
df.printSchema()

root
 |-- Time: integer (nullable = true)
 |-- SourceUserDomain: string (nullable = true)
 |-- DestinationUserDomain: string (nullable = true)
 |-- SourceComputer: string (nullable = true)
 |-- DestinationComputer: string (nullable = true)
 |-- AuthenticationType: string (nullable = true)
 |-- LogonType: string (nullable = true)
 |-- AuthenticationOrientation: string (nullable = true)
 |-- SuccessFailure: string (nullable = true)



In [11]:
#Jupyter notebook
df.show(5)

+----+--------------------+---------------------+--------------+-------------------+------------------+---------+-------------------------+--------------+
|Time|    SourceUserDomain|DestinationUserDomain|SourceComputer|DestinationComputer|AuthenticationType|LogonType|AuthenticationOrientation|SuccessFailure|
+----+--------------------+---------------------+--------------+-------------------+------------------+---------+-------------------------+--------------+
|   1|ANONYMOUS LOGON@C586| ANONYMOUS LOGON@C586|         C1250|               C586|              NTLM|  Network|                    LogOn|       Success|
|   1|ANONYMOUS LOGON@C586| ANONYMOUS LOGON@C586|          C586|               C586|                 ?|  Network|                   LogOff|       Success|
|   1|          C101$@DOM1|           C101$@DOM1|          C988|               C988|                 ?|  Network|                   LogOff|       Success|
|   1|         C1020$@DOM1|         SYSTEM@C1020|         C1020|      

In [19]:
df = df.withColumn('ID', sf.concat(sf.col('Time'),sf.col('DestinationComputer'), sf.col('SourceComputer'))).drop('joined_column')


df.show()

+----+--------------------+---------------------+--------------+-------------------+------------------+---------+-------------------------+--------------+-----------+
|Time|    SourceUserDomain|DestinationUserDomain|SourceComputer|DestinationComputer|AuthenticationType|LogonType|AuthenticationOrientation|SuccessFailure|         ID|
+----+--------------------+---------------------+--------------+-------------------+------------------+---------+-------------------------+--------------+-----------+
|   1|ANONYMOUS LOGON@C586| ANONYMOUS LOGON@C586|         C1250|               C586|              NTLM|  Network|                    LogOn|       Success| 1C586C1250|
|   1|ANONYMOUS LOGON@C586| ANONYMOUS LOGON@C586|          C586|               C586|                 ?|  Network|                   LogOff|       Success|  1C586C586|
|   1|          C101$@DOM1|           C101$@DOM1|          C988|               C988|                 ?|  Network|                   LogOff|       Success|  1C988C988

In [25]:
#write df to csv
df.write.csv('sampleclean.csv')

In [23]:
rdf = spark.read.csv("redteam.txt", header=True, inferSchema=True).toDF("Time","UserDomain","SourceComputer","DestinationComputer")

In [28]:
from pyspark.sql.functions import lit
rdf = rdf.withColumn('ID', sf.concat(sf.col('Time'),sf.col('DestinationComputer'), sf.col('SourceComputer'))).withColumn("redteam", lit(True))
rdf.show()

+------+----------+--------------+-------------------+-----------------+-------+
|  Time|UserDomain|SourceComputer|DestinationComputer|               ID|redteam|
+------+----------+--------------+-------------------+-----------------+-------+
|151036| U748@DOM1|        C17693|               C305| 151036C305C17693|   true|
|151648| U748@DOM1|        C17693|               C728| 151648C728C17693|   true|
|151993|U6115@DOM1|        C17693|              C1173|151993C1173C17693|   true|
|153792| U636@DOM1|        C17693|               C294| 153792C294C17693|   true|
|155219| U748@DOM1|        C17693|              C5693|155219C5693C17693|   true|
|155399| U748@DOM1|        C17693|               C152| 155399C152C17693|   true|
|155460| U748@DOM1|        C17693|              C2341|155460C2341C17693|   true|
|155591| U748@DOM1|        C17693|               C332| 155591C332C17693|   true|
|156658| U748@DOM1|        C17693|              C4280|156658C4280C17693|   true|
|210086| U748@DOM1|        C

In [26]:
#write rdf to csv
rdf.write.csv('redsampleclean.csv')